# Gestructureerde data: inladen dataset

## Enkel datatype

Er zijn heel veel verschillende bronnen om datasets te vinden. Je kan ze bijvoorbeeld op keras, overheidssites en dergelijke vinden.
Ook zijn er heel wat datasets die standaard geworden zijn voor voorbeelden/modellen te testen. 
Deze datasets worden mee aangeleverd met de verschillende modelling-frameworks en kunnen zo eenvoudig gebruikt worden zonder een extra download uit te voeren.
In deze notebook gaan we werken met deze standaard datasets.

In het eerste voorbeeld gaan we werken met een dataset waarbij alle data reeds numeriek is, meer bepaald gaan we werken met de iris-dataset dat informatie over de bloemen van een aantal iris-soorten bevat.

In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Pytorch

Om een dataset in lezen en klaar te maken in pytorch moet je werken met een Dataset. Dit kan je doen als volgt:

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader

# Define custom PyTorch Dataset
class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.long)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

### Keras / Tensorflow

Keras is vooral bedoeld als uitvoerend framework en niet om data in te laden. Hierdoor moeten we dus eerder werken met een tensorflow structuur om data in te laden.
In het geval van Tesnsorflow zijn er een aantal manieren om te werken met gestructureerde data.
De belangrijkste zijn in het geval van gestructureerde data zijn:
* Rechstreeks werken met numpy array/dataframes X en y (niet hier getoond want geen extra code nodig)
* from_tensor_slices: de dataset reeds ingelezen in dataframe/numpy array en gepreprocessed in een ander framework
* make_csv_dataset: Lees rechstreeks een csv in (belangrijk om het aantal epochs hier ook in te stellen (anders loopt het continue)

In [18]:
from tensorflow.data import Dataset
from tensorflow.data.experimental import make_csv_dataset

# methode: from_tensor_slices
train_ds = Dataset.from_tensor_slices((X_train, y_train)).batch(32)
test_ds = Dataset.from_tensor_slices((X_test, y_test)).batch(32)

# methode: lees csv
iris_ds = make_csv_dataset(
    'iris.csv',
    batch_size=5, # Klein om voorbeelden eenvoudiger te maken
    label_name='variety',
    num_epochs=1) # Voeg dit zeker toe

for batch, label in iris_ds.take(1):
  for key, value in batch.items():
    print(f"{key:20s}: {value}")
  print()
  print(f"{'label':20s}: {label}")

sepal.length        : [5.5 4.8 6.1 7.3 7. ]
sepal.width         : [4.2 3.  2.6 2.9 3.2]
petal.length        : [1.4 1.4 5.6 6.3 4.7]
petal.width         : [0.2 0.3 1.4 1.8 1.4]

label               : [b'Setosa' b'Setosa' b'Virginica' b'Virginica' b'Versicolor']


2024-07-24 08:50:06.940090: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Mixed types

Bovenstaande voorbeelden zijn eenvoudige voorbeelden omdat de data in principe reeds gepreprocessed is of uit slechts 1 datatype bestaat. 
Dit heeft als gevolg dat alle inputs op dezelfde manier verwerkt kunnen worden en er dus geen onderscheid gemaakt moet worden tussen verschillende kolommen.
Data met hetzelfde type kan dus eenvoudig aan een sequentieel neuraal netwerk model gepresenteerd worden.

Indien de dataset echter meerdere types bevatten kan de data niet rechtstreeks aan het neuraal netwerk doorgegeven worden aangezien dit type model enkel met numerieke data kan werken. 
Een standaard dataset waarin dit gebeurd is de titanic dataset.
Hoe dit gebeurd in de verschillende frameworks zie je hieronder

### Pytorch

Aangezien we data binnenkrijgen in een dataset als numpy array/dataframe kunnen we in de constructor of de get_item functie de nodige preprocessing stappen uitvoeren.

In [34]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

# Load Titanic dataset
titanic = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")

# Define custom PyTorch Dataset
class TitanicDataset(Dataset):
    def __init__(self, dataframe):
        # Preprocess data: handle missing values and convert categorical data
        dataframe.fillna({'age': dataframe['age'].median()}, inplace=True)
        dataframe.fillna({'embark_town': dataframe['embark_town'].mode()[0]}, inplace=True)
        dataframe['sex'] = dataframe['sex'].astype('category').cat.codes
        dataframe['class'] = dataframe['class'].astype('category').cat.codes
        dataframe['embark_town'] = dataframe['embark_town'].astype('category').cat.codes
        
        # Select relevant columns
        X = dataframe[['class', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'embark_town']]
        y = dataframe['survived']
        
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.long)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create the dataset
dataset = TitanicDataset(titanic)

# Split dataset into training and testing datasets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Keras/Tensorflow

Dit stukje is de meest complexe vorm om data in te laden maar misschien ook het meest flexible.
We gaan namelijk van een dataset met mixed types een sequentieel model maken waarbij de nodige preprocessing stappen aanwezig zijn in het model.
Dit gaat uiteindelijk leiden tot een vrij complexe modelopbouw.

Omdat preprocessing functionaliteiten slechts in de volgende stap gaan bestudeerd worden gaan we hier enkel kijken hoe we de data kunnen klaar zetten.
De twee methodes hiervoor zijn:
* Plaats elke input/feature in een apart Input object. Dit zijn symbolische tensors die gebruikt worden als placeholder voor onze data
* Maak een csv dataset (identiek als hierboven)

In [45]:
import tensorflow as tf
from keras import Input

titanic_features = titanic.copy()
titanic_labels = titanic_features.pop('survived')

inputs = {}

for name, column in titanic_features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = Input(shape=(1,), name=name, dtype=dtype)

inputs

{'sex': <KerasTensor shape=(None, 1), dtype=float32, sparse=None, name=sex>,
 'age': <KerasTensor shape=(None, 1), dtype=float32, sparse=None, name=age>,
 'n_siblings_spouses': <KerasTensor shape=(None, 1), dtype=float32, sparse=None, name=n_siblings_spouses>,
 'parch': <KerasTensor shape=(None, 1), dtype=float32, sparse=None, name=parch>,
 'fare': <KerasTensor shape=(None, 1), dtype=float32, sparse=None, name=fare>,
 'class': <KerasTensor shape=(None, 1), dtype=float32, sparse=None, name=class>,
 'deck': <KerasTensor shape=(None, 1), dtype=string, sparse=None, name=deck>,
 'embark_town': <KerasTensor shape=(None, 1), dtype=float32, sparse=None, name=embark_town>,
 'alone': <KerasTensor shape=(None, 1), dtype=string, sparse=None, name=alone>}

In [47]:
# de numerieke inputs kunnen dan als volgt geselecteerd worden
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

In [52]:
titanic_file_path = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic_ds = tf.data.experimental.make_csv_dataset(
    titanic_file_path,
    batch_size=5, 
    label_name='survived',
    num_epochs=1)

for batch, label in titanic_ds.take(1):
  for key, value in batch.items():
    print(f"{key:20s}: {value}")
  print()
  print(f"{'label':20s}: {label}")

sex                 : [b'male' b'female' b'male' b'male' b'female']
age                 : [36. 24. 29. 50. 29.]
n_siblings_spouses  : [0 0 1 1 0]
parch               : [0 0 0 0 0]
fare                : [  0.      69.3      7.0458 106.425  211.3375]
class               : [b'Third' b'First' b'Third' b'First' b'First']
deck                : [b'unknown' b'B' b'unknown' b'C' b'B']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'y' b'y' b'n' b'n' b'y']

label               : [0 1 0 0 1]


2024-07-24 09:26:23.438416: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
